In [1]:
"""Source:
https://www.kaggle.com/code/daehunbae/esm-2-pseudo-perplexity-ranking
"""

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import transformers
# from transformers import EsmTokenizer, EsmForMaskedLM
from transformers import ESMForMaskedLM, ESMTokenizer
import torch


In [2]:
df = pd.read_csv('../../all_4_results.csv')
df['ID'] = df['name'].apply(lambda x: '_'.join(x.split('_')[:2]))

In [ ]:

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    
    # mask one by one except [CLS] and [SEP]
    mask = torch.ones(tensor_input.size(-1) -1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    
    labels = repeat_input.masked_fill(masked_input != tokenizer.mask_token_id, -100)
    with torch.no_grad():
        loss = model(masked_input.to(device), labels=labels.to(device)).loss
    return np.exp(loss.item())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.chdir('/home/jupyter-yehlin/DMSV2/dataset_final')

tokenizer = ESMTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")
model = ESMForMaskedLM.from_pretrained("facebook/esm2_t33_650M_UR50D").to(device)
new_df = pd.DataFrame(['esm_pp'])
processed_names = set()

for index, row in df.iterrows():
    name = row['name']
    if name not in processed_names:
        score_val = score(model=model, tokenizer=tokenizer, sentence=row['sequence'])
        row['esm_pp'] = score_val
        print(row['esm_pp'])
        new_df = new_df.append(row, ignore_index=True)
        
new_df.to_csv('dmsv2_aa_seq_esm.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'EsmTokenizer'. 
The class this function is called from is 'ESMTokenizer'.
Some weights of the model checkpoint at facebook/esm2_t33_650M_UR50D were not used when initializing ESMForMaskedLM: ['esm.encoder.layer.1.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.8.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.24.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.19.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.21.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.11.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.2.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.7.attention.self.rotary_embeddings.inv_freq', 'esm.encoder.layer.13.attention.self.rotary_embeddings.inv_freq', 'esm.en

24.18810807890835
19.009342207133137
37.48652277940098
22.388505672304106
18.727747837636116
29.46407010193305
42.21633155663543
19.744756912578193
